In [ ]:
import os
import mysql.connector
from mysql.connector import Error
import pandas as pd
from dotenv import load_dotenv

# download_table

In [ ]:
# Load environment variables from .env file
# Make sure your .env file is in the same directory as your notebook or Python script,
# or provide the full path to load_dotenv()
load_dotenv()

def get_db_config():
    """Get database configuration from environment variables."""
    return {
        'host': os.getenv('DB_HOST', 'localhost'),
        'user': os.getenv('DB_USERNAME'),
        'password': os.getenv('DB_PASS'),
        'database': os.getenv('DB_NAME'),
        'port': int(os.getenv('DB_PORT', '3306')),
        'charset': os.getenv('DB_CHARSET', 'utf8mb4'),
        'collation': os.getenv('DB_COLLATION', 'utf8mb4_general_ci')
    }

def create_connection():
    """Create a database connection from .env configuration."""
    connection = None
    db_config = get_db_config()
    try:
        connection = mysql.connector.connect(**db_config)
        if connection.is_connected():
            print(f"Successfully connected to MariaDB: {db_config['user']}@{db_config['host']}/{db_config['database']}")
            return connection
    except Error as e:
        print(f"Error connecting to MariaDB: {e}")
        print("\nConnection details (excluding password):")
        safe_config = {k: v for k, v in db_config.items() if k != 'password'}
        print(safe_config)
        return None

def list_tables():
    """List all available tables in the database. Creates and closes its own connection."""
    connection = create_connection()
    if not connection:
        # print("Failed to connect to database for listing tables.") # create_connection already prints
        return None
    
    cursor = None
    table_names_list = None
    try:
        cursor = connection.cursor()
        cursor.execute("SHOW TABLES")
        tables_data = cursor.fetchall()
        
        if not tables_data:
            print("No tables found in the database.")
            table_names_list = []
        else:
            print("\nAvailable tables:")
            table_names_list = [table[0] for table in tables_data]
            for i, table_name_item in enumerate(table_names_list, 1):
                print(f"{i}. {table_name_item}")
        
        return table_names_list
    
    except Error as e:
        print(f"Error listing tables: {e}")
        return None
    finally:
        if connection and connection.is_connected():
            if cursor:
                cursor.close()
            connection.close()

def _list_all_tables_internal(connection):
    """
    List all available tables using an existing connection. Internal use.
    Returns a list of table names, or None on error.
    """
    cursor = None
    try:
        cursor = connection.cursor()
        cursor.execute("SHOW TABLES")
        tables_data = cursor.fetchall()
        if not tables_data:
            return []
        return [table[0] for table in tables_data]
    except Error as e:
        print(f"Error listing tables internally: {e}")
        return None
    finally:
        if cursor:
            cursor.close()

def _download_single_table_to_df_internal(connection, table_name):
    """
    Fetches all data from a single table and returns it as a Pandas DataFrame.
    Uses an existing connection. Internal use.
    Returns DataFrame on success (can be empty if table has no rows).
    Returns None if table doesn't exist or a database error occurs.
    """
    cursor = None
    try:
        cursor = connection.cursor(dictionary=True)
        cursor.execute(f"SHOW TABLES LIKE %s", (table_name,))
        if not cursor.fetchone():
            print(f"Table '{table_name}' does not exist in the database.")
            return None
        
        query = f"SELECT * FROM `{table_name}`"
        cursor.execute(query)
        rows = cursor.fetchall()
        
        if not rows:
            print(f"Table '{table_name}' is empty.")
            return pd.DataFrame()
        
        df = pd.DataFrame(rows)
        print(f"Fetched {len(df)} rows from '{table_name}'.")
        return df
        
    except Error as e:
        print(f"Error fetching data for table '{table_name}': {e}")
        return None
    finally:
        if cursor:
            cursor.close()

def download_data(tables_to_download, output_dir=".", output_format='csv'):
    """
    Download specified table(s) from MariaDB.

    Args:
        tables_to_download:
            - str: Name of a single table.
            - list: A list of table names.
            - 'all': String literal to download all tables.
        output_dir (str): Directory to save CSV files. Defaults to current directory.
                          Ignored if output_format is 'pandas'. Created if it doesn't exist for CSV.
        output_format (str): 'csv' or 'pandas'.

    Returns:
        - If output_format is 'csv':
            - True if all requested operations were successful, False otherwise.
        - If output_format is 'pandas':
            - For a single table request: DataFrame if successful, None otherwise.
            - For multiple tables or 'all': Dict[str, DataFrame]. Empty dict if no tables
              were processed/found or all failed. Individual table entries might be missing if
              that specific table failed.
        - Special case for pandas: If initial connection fails, returns None for single table request,
          or an empty dict for multiple/'all' requests.
    """
    connection = create_connection()
    if not connection:
        # print("Failed to connect to database for downloading data.") # create_connection prints
        if output_format == 'pandas':
            is_single_req = isinstance(tables_to_download, str) and tables_to_download.lower() != 'all'
            return None if is_single_req else {}
        return False

    results_dfs = {} if output_format == 'pandas' else None
    overall_success = True 
    
    table_names_to_process = []
    is_single_table_request = False

    try:
        if isinstance(tables_to_download, str) and tables_to_download.lower() == 'all':
            # print("Action: Attempting to download all tables.")
            fetched_tables = _list_all_tables_internal(connection)
            if fetched_tables is None: 
                print("Error: Failed to retrieve list of all tables from the database.")
                overall_success = False
            elif not fetched_tables:
                print("Info: No tables found in the database to download.")
            else:
                table_names_to_process = fetched_tables
        elif isinstance(tables_to_download, list):
            if not tables_to_download:
                print("Info: No tables specified in the list to download.")
            # else:
                # print(f"Action: Attempting to download tables: {', '.join(tables_to_download)}")
            table_names_to_process = tables_to_download
        elif isinstance(tables_to_download, str):
            # print(f"Action: Attempting to download table: '{tables_to_download}'")
            table_names_to_process = [tables_to_download]
            is_single_table_request = True
        else:
            print("Error: Invalid 'tables_to_download' argument. Must be a table name (str), list of names, or 'all'.")
            overall_success = False

        if not overall_success:
            if output_format == 'pandas':
                return None if is_single_table_request else {}
            return False

        if not table_names_to_process and overall_success: 
            if output_format == 'pandas':
                 return None if is_single_table_request else {}
            return True 

        if output_format == 'csv' and table_names_to_process: # Create dir only if CSV and tables exist
            os.makedirs(output_dir, exist_ok=True)

        for table_name in table_names_to_process:
            print(f"\nProcessing table: '{table_name}'...")
            df = _download_single_table_to_df_internal(connection, table_name)

            if df is not None: 
                if output_format == 'csv':
                    if not df.empty:
                        filename = os.path.join(output_dir, f"{table_name}.csv")
                        try:
                            df.to_csv(filename, index=False)
                            print(f"Data from '{table_name}' saved to {filename}")
                        except Exception as e:
                            print(f"Error saving table '{table_name}' to CSV: {e}")
                            overall_success = False
                elif output_format == 'pandas':
                    results_dfs[table_name] = df
            else: 
                overall_success = False 
                if is_single_table_request and output_format == 'csv':
                    break 

    except Error as e: 
        print(f"A database error occurred during the download process: {e}")
        overall_success = False
    except Exception as e: 
        print(f"An unexpected error occurred: {e}")
        overall_success = False
    finally:
        if connection and connection.is_connected():
            connection.close()
            print("\nDatabase connection (for downloading data) closed.")

    if output_format == 'pandas':
        if is_single_table_request:
            if table_names_to_process: 
                return results_dfs.get(table_names_to_process[0])
            return None 
        return results_dfs 
    else: 
        return overall_success


In [ ]:

# ==============================================================================
# Example Usage for IPython Notebook (copy and paste cells)
# ==============================================================================
#
# **Instructions for Notebook:**
# 1. Place this entire script (including the functions above) into a single
#    notebook cell and run it. This defines all the necessary functions.
# 2. In subsequent cells, you can use the example blocks below.
# 3. Ensure your .env file is in the same directory as your notebook or
#    provide the full path to `load_dotenv()` at the top.
# 4. Modify `TABLE_NAME_SINGLE`, `TABLE_NAMES_MULTIPLE`, and `CSV_DIR` as needed.
#
# ------------------------------------------------------------------------------

# --- Configuration (set these once in a cell if you like) ---
TABLE_NAME_SINGLE = "your_single_table"  # <<< REPLACE with an actual table name
TABLE_NAMES_MULTIPLE = ["your_table1", "your_table2"] # <<< REPLACE
CSV_DIR = "downloaded_db_tables_csv"

# --- Example 1: List all tables ---
# tables = list_tables()
# if tables is not None: # Check if listing was successful (not None)
#     print(f"\nAvailable tables: {tables if tables else 'None'}")

# --- Example 2: Download a SINGLE table to CSV ---
# print(f"\nDownloading '{TABLE_NAME_SINGLE}' to CSV in '{CSV_DIR}'...")
# success = download_data(TABLE_NAME_SINGLE, output_dir=CSV_DIR, output_format='csv')
# print(f"Result: {'Success' if success else 'Failed'}")

# --- Example 3: Download a SINGLE table to Pandas DataFrame ---
# print(f"\nDownloading '{TABLE_NAME_SINGLE}' to Pandas DataFrame...")
# df_one_table = download_data(TABLE_NAME_SINGLE, output_format='pandas')
# if df_one_table is not None:
#     print(f"DataFrame for '{TABLE_NAME_SINGLE}' has {len(df_one_table)} rows.")
#     # display(df_one_table.head()) # In a notebook, 'display()' is often preferred for DataFrames
# else:
#     print(f"Failed to get DataFrame for '{TABLE_NAME_SINGLE}'.")

# --- Example 4: Download MULTIPLE specified tables to CSV ---
# print(f"\nDownloading {TABLE_NAMES_MULTIPLE} to CSV files in '{CSV_DIR}'...")
# success_multiple = download_data(TABLE_NAMES_MULTIPLE, output_dir=CSV_DIR, output_format='csv')
# print(f"Result: {'Overall/Partial Success' if success_multiple else 'Overall Failed'}. Check logs.")

# --- Example 5: Download MULTIPLE specified tables to Pandas DataFrames ---
# print(f"\nDownloading {TABLE_NAMES_MULTIPLE} to Pandas DataFrames...")
# dict_of_dfs = download_data(TABLE_NAMES_MULTIPLE, output_format='pandas')
# if dict_of_dfs: # Check if dictionary is not empty
#     print(f"Received {len(dict_of_dfs)} DataFrame(s):")
#     for name, df in dict_of_dfs.items():
#         print(f"  - '{name}': {len(df)} rows")
#         # display(df.head(2)) # Optionally display head of each
# else:
#     print("No DataFrames received or all failed.")

# --- Example 6: Download ALL tables to CSV ---
print(f"\nDownloading ALL tables to CSV in '{CSV_DIR}/all_tables'...")
all_tables_csv_path = os.path.join(CSV_DIR, "all_tables")
success_all = download_data('all', output_dir=all_tables_csv_path, output_format='csv')
print(f"Result: {'Overall/Partial Success' if success_all else 'Overall Failed'}. Check logs.")

# --- Example 7: Download ALL tables to Pandas DataFrames ---
# print(f"\nDownloading ALL tables to Pandas DataFrames...")
# all_tables_dfs = download_data('all', output_format='pandas')
# if all_tables_dfs:
#     print(f"Received {len(all_tables_dfs)} DataFrame(s) for all tables:")
#     for name, df in all_tables_dfs.items():
#         print(f"  - '{name}': {len(df)} rows")
# else:
#     print("No DataFrames received or database is empty/all failed.")

# --- Example 8: Test non-existent table (Pandas) ---
# print(f"\nDownloading non-existent table 'fantasy_table_123' to Pandas...")
# df_non_existent = download_data("fantasy_table_123", output_format='pandas')
# if df_non_existent is None:
#     print("Correctly returned None for non-existent table.")
# else:
#     print("Unexpected: Did not return None for non-existent table.")

# Upload

In [ ]:
import os
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_db_config():
    """Get database configuration from environment variables."""
    return {
        'host': os.getenv('DB_HOST', 'localhost'),
        'user': os.getenv('DB_USERNAME'),
        'password': os.getenv('DB_PASS'),
        'database': os.getenv('DB_NAME'),
        'port': int(os.getenv('DB_PORT', '3306')),
        'charset': os.getenv('DB_CHARSET', 'utf8mb4'),
        'collation': os.getenv('DB_COLLATION', 'utf8mb4_general_ci')
    }

def create_connection():
    """Create a database connection from .env configuration."""
    connection = None
    try:
        db_config = get_db_config()
        connection = mysql.connector.connect(**db_config)
        if connection.is_connected():
            print(f"Successfully connected to MariaDB at {db_config['host']}")
            print(f"Connected to server version {connection.get_server_info()}")
            print(f"Database: {db_config['database']}")
            return connection
    except Error as e:
        print(f"Error connecting to MariaDB: {e}")
        print("\nConnection details (excluding password):")
        safe_config = {k: v for k, v in get_db_config().items() if k != 'password'}
        print(safe_config)
        return None

def upload_csv_to_db(csv_file_path, table_name, if_exists='replace', chunk_size=None):
    """Upload a CSV file to a MariaDB table with improved handling for large tables.
    
    Args:
        csv_file_path: Path to the CSV file
        table_name: Name of the table to create or use
        if_exists: What to do if the table already exists ('replace', 'append', 'fail')
        chunk_size: Process CSV in chunks of this size (None to load entire file)
        
    Returns:
        bool: True if successful, False otherwise
    """
    connection = create_connection()
    if not connection:
        print("Failed to connect to database")
        return False
    
    cursor = None
    try:
        # First, sample the data to understand its structure
        # Read a sample to analyze column types
        sample_size = 1000  # Adjust based on your data size
        sample_df = pd.read_csv(csv_file_path, nrows=sample_size)
        
        print(f"Detected {len(sample_df.columns)} columns in CSV file")
        
        # Create a dictionary to store column types and a mapping for column names
        column_types = {}
        column_name_mapping = {}
        
        # Create mapping from original column names to clean column names
        for column in sample_df.columns:
            clean_column = ''.join(e for e in column if e.isalnum() or e == '_')
            column_name_mapping[column] = clean_column
        
        # Rename columns in the sample dataframe
        sample_df.rename(columns=column_name_mapping, inplace=True)
        
        # Preprocess the sample data to get a better understanding of column types
        for column in sample_df.columns:
            # Check if column contains any non-numeric values
            if sample_df[column].dtype == 'object':
                # If it's a string column, check if it could be numeric
                try:
                    # Try to convert to numeric
                    pd.to_numeric(sample_df[column], errors='raise')
                    # If successful, it's likely a numeric column
                    column_types[column] = "FLOAT"
                except:
                    # If conversion fails, it contains non-numeric values
                    column_types[column] = "TEXT"
            elif pd.api.types.is_integer_dtype(sample_df[column].dtype):
                column_types[column] = "INT"
            elif pd.api.types.is_float_dtype(sample_df[column].dtype):
                column_types[column] = "FLOAT"
            elif pd.api.types.is_bool_dtype(sample_df[column].dtype):
                column_types[column] = "BOOLEAN"
            elif pd.api.types.is_datetime64_any_dtype(sample_df[column].dtype):
                column_types[column] = "DATETIME"
            else:
                column_types[column] = "TEXT"
        
        # Now, process the entire file or in chunks
        if chunk_size:
            # Process in chunks
            df_iterator = pd.read_csv(csv_file_path, chunksize=chunk_size)
            first_chunk = next(df_iterator)
            # Rename columns in first chunk
            first_chunk.rename(columns=column_name_mapping, inplace=True)
            df_chunks = [first_chunk]
            
            # Rename columns in remaining chunks
            remaining_chunks = []
            for chunk in df_iterator:
                chunk.rename(columns=column_name_mapping, inplace=True)
                remaining_chunks.append(chunk)
            
            df_chunks.extend(remaining_chunks)
        else:
            # Read the entire CSV file
            df = pd.read_csv(csv_file_path)
            print(f"Read {len(df)} rows from {csv_file_path}")
            # Rename columns in the full dataframe
            df.rename(columns=column_name_mapping, inplace=True)
            df = df.replace({np.nan: None})
        
        cursor = connection.cursor()
        
        # Check if table exists
        cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
        table_exists = cursor.fetchone() is not None
        
        if table_exists:
            if if_exists == 'fail':
                print(f"Table '{table_name}' already exists. Aborting.")
                return False
            elif if_exists == 'replace':
                print(f"Dropping existing table '{table_name}'")
                cursor.execute(f"DROP TABLE {table_name}")
                connection.commit()
                table_exists = False
        
        # Create table if it doesn't exist
        if not table_exists:
            # Use the column types we detected earlier
            column_defs = []
            
            # Process each column to create the table definition
            for column in sample_df.columns:
                sql_type = column_types.get(column, "TEXT")  # Default to TEXT if not found
                
                # Special handling for MoveIn column which caused the error in the past
                if column == 'MoveIn':
                    sql_type = "TEXT"  # Force TEXT for MoveIn
                
                column_defs.append(f"`{column}` {sql_type}")
            
            # Create table
            create_table_sql = f"CREATE TABLE {table_name} ({', '.join(column_defs)})"
            print(f"Creating table with SQL: {create_table_sql}")
            cursor.execute(create_table_sql)
            connection.commit()
            print(f"Table '{table_name}' created successfully")
        
        # Insert data
        def insert_dataframe(df_to_insert):
            # Prepare the SQL placeholders and column names
            placeholders = ', '.join(['%s'] * len(df_to_insert.columns))
            columns = ', '.join(f'`{col}`' for col in df_to_insert.columns)
            
            insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            
            # Convert DataFrame to list of tuples
            values = df_to_insert.replace({np.nan: None}).values.tolist()
            
            # Execute in batches to avoid memory issues with large datasets
            batch_size = 1000
            for i in range(0, len(values), batch_size):
                batch = values[i:i+batch_size]
                cursor.executemany(insert_sql, batch)
                connection.commit()
                print(f"Inserted batch {i//batch_size + 1} ({len(batch)} rows)")
        
        # Insert data from chunks or entire dataframe
        if chunk_size:
            total_rows = 0
            for i, chunk in enumerate(df_chunks):
                insert_dataframe(chunk)
                total_rows += len(chunk)
                print(f"Processed chunk {i+1} with {len(chunk)} rows")
            print(f"Successfully uploaded {total_rows} rows to table '{table_name}'")
        else:
            insert_dataframe(df)
            print(f"Successfully uploaded {len(df)} rows to table '{table_name}'")
        
        return True
    
    except Error as e:
        print(f"Error uploading CSV to database: {e}")
        return False
    except Exception as e:
        print(f"General error: {e}")
        import traceback
        traceback.print_exc()
        return False
    finally:
        if cursor:
            cursor.close()
        if connection and connection.is_connected():
            connection.close()
            print("Database connection closed")

# Example usage
if __name__ == "__main__":
    csv_file = r"d:\OneDrive - Green Energy\Desktop\properties_202503181522.csv"  # Replace with your CSV file path
    table_name = "property"  # Replace with your desired table name
    upload_csv_to_db(csv_file, table_name, chunk_size=50)  # Process in chunks of 50 rows